In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_validate

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV 
from scipy.stats import randint

from sklearn.model_selection import KFold 
from sklearn.model_selection import StratifiedKFold 
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import RepeatedKFold 
from sklearn.model_selection import RepeatedStratifiedKFold

from sklearn.preprocessing import MinMaxScaler

from sklearn.pipeline import Pipeline

In [3]:
data_music=pd.read_excel('feature_extraction_3.xls')
data_music.head()

,Unnamed: 0,filename,label,fsc_mean,fsc_var,fsb_mean,fsb_var,chroma_mean,chroma_var,rms_mean,...,mfcc_mean21,mfcc_mean22,mfcc_mean23,mfcc_mean24,mfcc_mean25,mfcc_mean26,mfcc_mean27,mfcc_mean28,mfcc_mean29,mfcc_mean30
0,0,blues.000000.wav,blues,1775.747689,168218.236006,1976.307347,113785.200935,0.412760,0.124600,0.130077,...,-5.577494,-0.806646,-1.833445,-0.998209,2.528299,2.980887,-3.815330,-4.999353,-0.248520,1.828161
1,1,blues.000001.wav,blues,1782.417617,90849.067327,2003.793417,59525.192006,0.382884,0.126445,0.108474,...,-1.446377,0.119136,-4.970406,-2.800108,2.132017,1.239747,-7.189402,-6.137180,-1.379605,1.629855
2,2,blues.000002.wav,blues,1810.274575,90165.503975,2091.004353,77975.685006,0.312193,0.084679,0.137133,...,-6.599291,-2.585352,-4.982160,-5.021442,0.301683,-1.802398,-6.943006,-3.287982,3.770696,3.792954
3,3,blues.000003.wav,blues,1652.650516,103062.026806,1968.470394,68974.341182,0.331422,0.090406,0.137032,...,-2.986648,0.210857,-6.007319,-2.055582,-1.174838,0.938859,-8.474980,-1.422811,3.097076,4.958448
4,4,blues.000004.wav,blues,1821.400645,212730.331504,2037.861849,93774.984788,0.306959,0.088976,0.141036,...,-4.165588,-0.844202,-6.071796,-5.429046,-1.830063,-1.235257,-5.226347,-1.414961,2.900378,2.612174


In [4]:
#splitting the label and features into 2 dataframes
X = data_music.iloc[:,3:]
Y = data_music['label']


In [5]:
#splitting into train and tesy
train_set, test_set, train_labels, test_labels = train_test_split(X, Y,test_size=0.2, random_state=8)


In [6]:
# normalise the data 
min_max_scaler = MinMaxScaler()
X_train_scaled = min_max_scaler.fit_transform(train_set)

### Models

##### Gaussian

In [13]:
nb = GaussianNB()
NB_scores = cross_val_score(nb,X_train_scaled,train_labels, cv=10)
NB_scores.mean()

0.522337453066333

##### Stochastic Gradient Descent

In [14]:
# Stochastic Gradient Descent
sgd = SGDClassifier()

param_dist = {
    'alpha': [1e-5,1e-4, 1e-3], # learning rate
    'max_iter': [3000,4000,5000], # number of epochs
    'loss': ['log'],
    'penalty': ['l1', 'elasticnet']
}

rnd_search = RandomizedSearchCV(sgd, param_distributions=param_dist,
                                n_iter=18, cv=10,n_jobs = -1)
rnd_search.fit(X_train_scaled,train_labels)

rand_cv_res = rnd_search.cv_results_
for mean_score, params in zip(rand_cv_res["mean_test_score"], rand_cv_res["params"]):
    print(f"score = {mean_score} <- parameters {params}")


score = 0.6895248748435544 <- parameters {'penalty': 'l1', 'max_iter': 3000, 'loss': 'log', 'alpha': 1e-05}
score = 0.6586156132665832 <- parameters {'penalty': 'elasticnet', 'max_iter': 3000, 'loss': 'log', 'alpha': 1e-05}
score = 0.6788868898623278 <- parameters {'penalty': 'l1', 'max_iter': 4000, 'loss': 'log', 'alpha': 1e-05}
score = 0.6634928035043804 <- parameters {'penalty': 'elasticnet', 'max_iter': 4000, 'loss': 'log', 'alpha': 1e-05}
score = 0.6827658010012516 <- parameters {'penalty': 'l1', 'max_iter': 5000, 'loss': 'log', 'alpha': 1e-05}
score = 0.6624924906132665 <- parameters {'penalty': 'elasticnet', 'max_iter': 5000, 'loss': 'log', 'alpha': 1e-05}
score = 0.6979081664580725 <- parameters {'penalty': 'l1', 'max_iter': 3000, 'loss': 'log', 'alpha': 0.0001}
score = 0.6792614205256571 <- parameters {'penalty': 'elasticnet', 'max_iter': 3000, 'loss': 'log', 'alpha': 0.0001}
score = 0.6940295682102627 <- parameters {'penalty': 'l1', 'max_iter': 4000, 'loss': 'log', 'alpha': 0

In [16]:
rnd_search.best_estimator_

SGDClassifier(loss='log', max_iter=3000, penalty='l1')

##### KNN

In [7]:

knn = KNeighborsClassifier()

param_dist = {'n_neighbors': list(range(1,30)),
              'leaf_size' : list(range(1,30)),
              'p': [1,2]
            }

rnd_search = RandomizedSearchCV(knn, param_distributions=param_dist,
                                n_iter=30, cv=10,n_jobs = -1)
rnd_search.fit(X_train_scaled,train_labels)

rand_cv_res = rnd_search.cv_results_
for mean_score, params in zip(rand_cv_res["mean_test_score"], rand_cv_res["params"]):
    print(f"score = {mean_score} <- parameters {params}")

score = 0.8179208385481852 <- parameters {'p': 2, 'n_neighbors': 15, 'leaf_size': 1}
score = 0.8742345118898622 <- parameters {'p': 1, 'n_neighbors': 8, 'leaf_size': 15}
score = 0.8544618272841051 <- parameters {'p': 2, 'n_neighbors': 8, 'leaf_size': 13}
score = 0.7977728410513142 <- parameters {'p': 2, 'n_neighbors': 19, 'leaf_size': 13}
score = 0.8813674906132667 <- parameters {'p': 1, 'n_neighbors': 6, 'leaf_size': 7}
score = 0.8998876720901127 <- parameters {'p': 1, 'n_neighbors': 3, 'leaf_size': 7}
score = 0.8454519712140176 <- parameters {'p': 2, 'n_neighbors': 9, 'leaf_size': 6}
score = 0.7698681163954945 <- parameters {'p': 2, 'n_neighbors': 28, 'leaf_size': 27}
score = 0.7656127972465582 <- parameters {'p': 2, 'n_neighbors': 29, 'leaf_size': 19}
score = 0.8574638610763454 <- parameters {'p': 1, 'n_neighbors': 11, 'leaf_size': 8}
score = 0.8764860763454319 <- parameters {'p': 2, 'n_neighbors': 4, 'leaf_size': 6}
score = 0.8333130475594495 <- parameters {'p': 1, 'n_neighbors': 1

In [8]:
rnd_search.best_estimator_

KNeighborsClassifier(leaf_size=7, n_neighbors=3, p=1)

##### Support Vector Machine

In [8]:
svm = SVC()

param_dist = {'kernel' : ['linear', 'rbf'],
              'C' : [0.1, 1, 10, 100, 1000],
              'degree' : [0, 1, 2, 3, 4, 5, 6],
              'decision_function_shape' :["ovo"]
                }

rnd_search = RandomizedSearchCV(svm, param_distributions=param_dist,
                                n_iter=15, cv=3,n_jobs = -1)
rnd_search.fit(X_train_scaled,train_labels)

rand_cv_res = rnd_search.cv_results_
for mean_score, params in zip(rand_cv_res["mean_test_score"], rand_cv_res["params"]):
    print(f"score = {mean_score} <- parameters {params}")

score = 0.7315730112388018 <- parameters {'kernel': 'linear', 'degree': 3, 'decision_function_shape': 'ovo', 'C': 1}
score = 0.7400836811276165 <- parameters {'kernel': 'linear', 'degree': 2, 'decision_function_shape': 'ovo', 'C': 1000}
score = 0.746715641664947 <- parameters {'kernel': 'linear', 'degree': 1, 'decision_function_shape': 'ovo', 'C': 100}
score = 0.7512206160347356 <- parameters {'kernel': 'linear', 'degree': 6, 'decision_function_shape': 'ovo', 'C': 10}
score = 0.7315730112388018 <- parameters {'kernel': 'linear', 'degree': 5, 'decision_function_shape': 'ovo', 'C': 1}
score = 0.8734835567824678 <- parameters {'kernel': 'rbf', 'degree': 0, 'decision_function_shape': 'ovo', 'C': 100}
score = 0.7315730112388018 <- parameters {'kernel': 'linear', 'degree': 2, 'decision_function_shape': 'ovo', 'C': 1}
score = 0.7512206160347356 <- parameters {'kernel': 'linear', 'degree': 0, 'decision_function_shape': 'ovo', 'C': 10}
score = 0.5911647659676199 <- parameters {'kernel': 'rbf', 

In [9]:
rnd_search.best_estimator_

SVC(C=100, decision_function_shape='ovo', degree=0)

In [ ]:
 LogisticRegression().get_params().keys()

##### LogisticRegression

In [12]:

lg = LogisticRegression()

param_dist = {'solver' : ['lbfgs', 'liblinear', 'sag', 'saga'],
              'penalty' : ['l2'],
              'C' : [100, 10, 1.0, 0.1, 0.01]
                }

rnd_search = RandomizedSearchCV(lg, param_distributions=param_dist,
                                n_iter=20, cv=3,n_jobs = -1)
rnd_search.fit(X_train_scaled,train_labels)

rand_cv_res = rnd_search.cv_results_
for mean_score, params in zip(rand_cv_res["mean_test_score"], rand_cv_res["params"]):
    print(f"score = {mean_score} <- parameters {params}")

C:\Users\design-cj\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:278: UserWarning: The total space of parameters 20 is smaller than n_iter=100. Running 20 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


score = 0.22713062292484 <- parameters {'solver': 'lbfgs', 'penalty': 'l2', 'C': 100}
score = 0.5527471707531789 <- parameters {'solver': 'liblinear', 'penalty': 'l2', 'C': 100}
score = 0.22550390232515657 <- parameters {'solver': 'sag', 'penalty': 'l2', 'C': 100}
score = 0.20410482769664143 <- parameters {'solver': 'saga', 'penalty': 'l2', 'C': 100}
score = 0.2277562485504656 <- parameters {'solver': 'lbfgs', 'penalty': 'l2', 'C': 10}
score = 0.5492419287481248 <- parameters {'solver': 'liblinear', 'penalty': 'l2', 'C': 10}
score = 0.22562902745028168 <- parameters {'solver': 'sag', 'penalty': 'l2', 'C': 10}
score = 0.20423004679482337 <- parameters {'solver': 'saga', 'penalty': 'l2', 'C': 10}
score = 0.22688037267458971 <- parameters {'solver': 'lbfgs', 'penalty': 'l2', 'C': 1.0}
score = 0.5600042870508511 <- parameters {'solver': 'liblinear', 'penalty': 'l2', 'C': 1.0}
score = 0.22562893347722487 <- parameters {'solver': 'sag', 'penalty': 'l2', 'C': 1.0}
score = 0.20410487468316985 

In [11]:
rnd_search.best_estimator_

LogisticRegression(solver='liblinear')

##### RandomForestClassifier

In [19]:
rf = RandomForestClassifier()



param_dist= {'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
            'max_features': ['auto', 'sqrt'],
            'min_samples_leaf': [1, 2, 4],
            'min_samples_split': [2, 5, 10],
            'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]
           }


rnd_search = RandomizedSearchCV(rf, param_distributions=param_dist,
                                n_iter=10, cv=2,n_jobs = -1)
rnd_search.fit(X_train_scaled,train_labels)

rand_cv_res = rnd_search.cv_results_
for mean_score, params in zip(rand_cv_res["mean_test_score"], rand_cv_res["params"]):
    print(f"score = {mean_score} <- parameters {params}")

score = 0.8120381958930144 <- parameters {'n_estimators': 1600, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': 90}
score = 0.818796205467044 <- parameters {'n_estimators': 1200, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 40}
score = 0.798773210006001 <- parameters {'n_estimators': 1200, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 70}
score = 0.8226752722685138 <- parameters {'n_estimators': 2000, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 20}
score = 0.8016511818451744 <- parameters {'n_estimators': 2000, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 70}
score = 0.7928915774347565 <- parameters {'n_estimators': 1000, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 100}
score = 0.7842563777795317 <- parameters {'n_estimators': 800, 'min_samples_split': 10,

In [21]:
rnd_search.best_estimator_

RandomForestClassifier(max_depth=20, n_estimators=2000)

## pipeline 

In [8]:
MPP = Pipeline([
        ('scaler', MinMaxScaler()),('KNN',KNeighborsClassifier(leaf_size=11, n_neighbors=1)),
    ])

## testing

In [10]:
MPP.fit(train_set , train_labels)
preds_train= MPP.predict(train_set)
print('Accuracy on Train ', "knn", ':', accuracy_score(train_labels, preds_train))
preds = MPP.predict(test_set)
print('Accuracy', "knn", ':', accuracy_score(test_labels, preds))

Accuracy on Train  knn : 0.9991240145163309
Accuracy knn : 0.8868868868868869
